<a href="https://colab.research.google.com/github/CrisMcode111/DI_Bootcamp/blob/main/w6_d4_Finetune_LLMs_with_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Daily Challenge : How to Finetune LLMs with LoRA


Parameter-Efficient Fine-Tuning (PEFT) methods, like LoRA, address the challenges of fine-tuning large language models (LLMs) by only updating a small subset of the model’s parameters. This approach significantly reduces computational and storage costs, making LLM fine-tuning more accessible. PEFT techniques allow developers to adapt pre-trained models to specific tasks without retraining the entire model, leading to faster development cycles and reduced resource consumption.
You will implement it for this challenge.



👩‍🏫 👩🏿‍🏫 What You’ll learn
How to apply Low-Rank Adaptation (LoRA) to a pre-trained language model.
How to fine-tune a LoRA-adapted model using the Hugging Face PEFT library.
How to save and load a fine-tuned LoRA model.
How to perform inference using a fine-tuned LoRA model.


🛠️ What you will create
A fine-tuned language model that generates text based on a specific dataset of quotes, using LoRA.


Dataset
The “Abirate/english_quotes” dataset, specifically a 10% sample of the training split.


Task
* Install necessary libraries (PEFT, datasets).
* Load a pre-trained language model (bigscience/bloomz-560m) and its tokenizer.
* Load the dataset and preprocess it for the model.
* Configure LoRA using LoraConfig.
* Apply LoRA to the pre-trained model using get_peft_model.
* Set up training arguments using TrainingArguments.
* Initialize and train the model using Trainer.
* Save the fine-tuned LoRA model.
* Load the saved LoRA model for inference using PeftModel.from_pretrained.
* Generate text using the fine-tuned model and the tokenizer.

In [1]:
%pip install peft==0.4.0

!pip install datasets

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1


In [2]:
# 1) Model + tokenizer
model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer # Import AutoTokenizer here
import peft
from peft import LoraConfig, get_peft_model

# Re-define model_name and tokenizer to ensure they are available
model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2) Dataset – 10% from the split of train

data = load_dataset("Abirate/english_quotes", split="train[:2%]")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_sample = data.select(range(5))
display(train_sample)

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 5
})

In [4]:
# 3) Config LoRA (r=1, target_modules for BLOOMZ)
lora_config = LoraConfig(
    r=1,
    lora_alpha=1,  # scaling factor, just simple
    target_modules=["query_key_value"],  # attention layers from BLOOMZ
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Re-define model_name, tokenizer, and foundation_model for robustness
model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)

# 4) we add LoRA adapters over the base model
peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 98,304 || all params: 559,312,896 || trainable%: 0.01757585078102687
None


In [ ]:
# 5) Trainer

import transformers
from transformers import TrainingArguments, Trainer
import os
import time

output_directory = os.path.join("../cache/working", "peft_lab_outputs")
training_args = TrainingArguments(
    report_to="none",
    output_dir=output_directory,
    auto_find_batch_size=True,
    learning_rate=3e-2,  # bigger then full fine-tuning as it asked
    num_train_epochs=1,
    use_cpu=True
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=data,  # not 'e', we use tokenised dataset
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer,
        mlm=False
    )
)

trainer.train()

Step,Training Loss


In [ ]:
# 6) Save LoRA model fine-tunat
time_now = str(int(time.time()))
peft_model_path = os.path.join(output_directory, f"peft_model_{time_now}")
trainer.model.save_pretrained(peft_model_path)



In [6]:
# 7) Generate text with fine-tune model

inputs = tokenizer("Two things are infinite: ", return_tensors="pt")
outputs = peft_model.generate(
    **inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50,
    top_p=0.95
)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Two things are infinite:  things and things']
